### Summary
This Script pulls data from the pilot

In [263]:
import pymysql
import pandas as pd

conn = pymysql.connect(
    host="thirdtest.camwsondhmqr.us-east-2.rds.amazonaws.com",
    user="thirdtest",
    passwd="Qja1998+0325",
    db="ebdb"
)
cursor = conn.cursor()

In [264]:
cursor.execute('SELECT * FROM user');
rows = cursor.fetchall()

df_users = pd.DataFrame( [[ij for ij in i] for i in rows] )
df_users.rename(
    columns={0:'Name', 1:'Political Stance', 2:'Session', 3:'Question Order', 4:'Current Question', 5:'World'},
    inplace=True
)
#df_users = df_users.sort_values(['Name'], ascending=[1])
df_users['Turn'] = range(1,len(df_users)+1)
df_users = df_users[['Turn','Name','Political Stance','Session', 'Question Order', 'World']]
df_users


,Turn,Name,Political Stance,Session,Question Order,World
0,1,A1MMC6X3ZNJ9OT,Democrats,5b7b19ba1a7f3,"3,1,18,5,21,9,2,14,10,16,8,17,4,13,22,19,15,6,...",2
1,2,AZOF6QSSWG98H,Republicans,5b7b379c66923,"5,11,17,3,21,19,18,2,6,1,10,8,13,20,22,7,16,12...",2
2,3,A125KW9P18V5Z1,Democrats,5b7b3b116d1f5,"7,17,18,2,21,10,4,5,1,3,20,16,9,6,22,14,15,12,...",2
3,4,AAXYYH9MI3PJM,Republicans,5b7b3d7761086,"17,1,9,4,21,10,5,8,11,15,19,13,7,2,22,16,18,6,...",2
4,5,A3NMQ3019X6YE0,Republicans,5b7b5ad072903,"14,18,6,8,21,7,19,11,5,15,2,16,17,9,22,1,3,4,2...",2
5,6,A2YM550H2AL6JO,Democrats,5b7b62986aaf7,"5,13,1,15,21,16,10,4,11,17,7,8,6,18,22,2,3,19,...",2
6,7,A259BAVW85T0NT,Democrats,5b7b69ff34936,"2,17,4,6,21,11,12,15,8,10,1,9,20,14,22,3,16,18...",2
7,8,A2Q1BZPQ1BFPSM,Republicans,5b7b6e3d907c5,"13,7,12,16,21,1,9,5,4,3,6,10,19,14,22,11,20,18...",2
8,9,A1Y6YA93AFCHPY,Republicans,5b7b734d253af,"18,11,8,6,21,10,19,15,7,12,2,14,3,4,22,9,1,17,...",2
9,10,A3VAAV7ZCSO7NA,Democrats,5b7b78f7c468c,"5,2,7,12,21,15,18,9,13,3,14,6,8,11,22,19,16,4,...",2


In [265]:
cursor.execute('SELECT * FROM user_question_world_answer WHERE world_id != 1');
rows = cursor.fetchall()

df_responses = pd.DataFrame( [[ij for ij in i] for i in rows] )
df_responses.rename(
    columns={0:'Name', 1:'World', 2:'Question', 3:'Response'},
    inplace=True
)
#df_responses = df_responses.sort_values(['Name'], ascending=[1])
df_responses = df_responses.pivot(index='Name', columns='Question', values='Response')

df_users.set_index('Name').join(df_responses).sort_values('Turn')

,Turn,Political Stance,Session,Question Order,World,1,2,3,4,5,...,13,14,15,16,17,18,19,20,21,22
Name,,,,,,,,,,,,,,,,,,,,,
A1MMC6X3ZNJ9OT,1,Democrats,5b7b19ba1a7f3,"3,1,18,5,21,9,2,14,10,16,8,17,4,13,22,19,15,6,...",2,oppose,support,oppose,oppose,support,...,support,support,support,support,oppose,oppose,oppose,support,support,oppose
AZOF6QSSWG98H,2,Republicans,5b7b379c66923,"5,11,17,3,21,19,18,2,6,1,10,8,13,20,22,7,16,12...",2,oppose,support,oppose,oppose,support,...,support,oppose,oppose,oppose,support,oppose,oppose,oppose,support,oppose
A125KW9P18V5Z1,3,Democrats,5b7b3b116d1f5,"7,17,18,2,21,10,4,5,1,3,20,16,9,6,22,14,15,12,...",2,oppose,oppose,support,support,oppose,...,support,support,support,oppose,oppose,oppose,oppose,support,support,oppose
AAXYYH9MI3PJM,4,Republicans,5b7b3d7761086,"17,1,9,4,21,10,5,8,11,15,19,13,7,2,22,16,18,6,...",2,support,oppose,oppose,oppose,support,...,support,oppose,oppose,support,oppose,support,oppose,oppose,support,oppose
A3NMQ3019X6YE0,5,Republicans,5b7b5ad072903,"14,18,6,8,21,7,19,11,5,15,2,16,17,9,22,1,3,4,2...",2,oppose,oppose,oppose,support,support,...,oppose,support,support,oppose,oppose,support,support,support,support,oppose
A2YM550H2AL6JO,6,Democrats,5b7b62986aaf7,"5,13,1,15,21,16,10,4,11,17,7,8,6,18,22,2,3,19,...",2,oppose,oppose,oppose,support,support,...,oppose,oppose,oppose,oppose,oppose,oppose,oppose,oppose,support,oppose
A259BAVW85T0NT,7,Democrats,5b7b69ff34936,"2,17,4,6,21,11,12,15,8,10,1,9,20,14,22,3,16,18...",2,oppose,oppose,oppose,oppose,support,...,support,oppose,oppose,support,oppose,oppose,oppose,support,support,oppose
A2Q1BZPQ1BFPSM,8,Republicans,5b7b6e3d907c5,"13,7,12,16,21,1,9,5,4,3,6,10,19,14,22,11,20,18...",2,oppose,oppose,oppose,oppose,oppose,...,oppose,oppose,oppose,support,support,oppose,support,oppose,support,support
A1Y6YA93AFCHPY,9,Republicans,5b7b734d253af,"18,11,8,6,21,10,19,15,7,12,2,14,3,4,22,9,1,17,...",2,oppose,oppose,oppose,oppose,oppose,...,support,oppose,oppose,support,oppose,oppose,support,support,support,oppose


In [266]:
df_users.set_index('Name').join(df_responses).sort_values('Turn').to_csv('pilot_data.csv')